### 将比赛TXT文件转换为CSV文件

In [ ]:
import pandas as pd
import numpy as np
import gc
df=pd.read_csv(r'D:\Data\IJCAI\round1_ijcai_18_test_a_20180301\round1_ijcai_18_test_a_20180301.txt',sep=' ')
df.to_csv(r'D:\Data\IJCAI\test.csv',index=False)
del df
gc.collect()
dff=pd.read_csv(r'D:\Data\IJCAI\round1_ijcai_18_train_20180301\round1_ijcai_18_train_20180301.txt',sep=' ')
dff.to_csv(r'D:\Data\IJCAI\train.csv',index=False)
del dff                           
gc.collect()

In [1]:
import os
j=os.listdir(r'C:\Users\13307\Desktop\新增未备份文件\IJCAI\github')
lis=[]
for i in j:
    lis.append(i)
lis

['.ipynb_checkpoints',
 'preprocess.ipynb',
 'Preprocess_no_fixed-newday.ipynb',
 'Preprocess_no_fixed.ipynb',
 'result.ipynb',
 'result_no-fixed-newday.ipynb',
 'result_no_fixed.ipynb',
 'untitled.txt',
 'Visualization.ipynb']

### 看一下item_category_list列可以拆成几列，每列需要多少数字，可以看到第一份是相同的，可以忽略

In [ ]:

j=0
listrain=[]
listest=[]
for data in combine:
    data['item_category_list_0']=0
    data['item_category_list_1']=0
    for ind in data.index:
        if j==0:
            if len(data.loc[ind,'item_category_list'].split(';'))==3:
                listrain.append(data.loc[ind,'item_category_list'].split(';')[2])
        else:
            if len(data.loc[ind,'item_category_list'].split(';'))==3:
                listest.append(data.loc[ind,'item_category_list'].split(';')[2])
    j=100
#set(listrain)
#set(listest)
h_=train['item_category_list'].apply(lambda x: x.split(';')[1])
h=test['item_category_list'].apply(lambda x: x.split(';')[1])
# dic={'1968056100269760729','2011981573061447208','22731265849056483','2436715285093487584','2642175453151805566','3203673979138763595','4879721024980945592', '509660095530134768','5755694407684602296','5799347067982556520','7258015885215914736','8277336076276184272','8710739180200009128'}
# h=test['item_category_list'].apply(lambda x: x.split(';')[1])
# dic={}
# count=0
# for i in set(h):
#     dic[i]=count
#     count+=1
# dict={'6233669177166538628':0, '8868887661186419229':1}

### 查看每列缺失值的数量和比率函数

In [ ]:
def missing(combine):
    mark=0
    for data in combine:
        dic={}
        dic['Alength']=len(data)
        for i in data.isnull().any()[data.isnull().any()].index:
            dic[i]=[str(len(data[data[i].isnull()]))+'   '+str(round(100*len(data[data[i].isnull()])/len(data),3))+'%']
        if mark==100:
            nantest=pd.DataFrame(dic)
            break
        else:
            nantrain=pd.DataFrame(dic)
        mark=100
    return nantrain,nantest

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\train.csv',na_values=-1)#,na_values=-1 [160955, 205287, 214183, 314706]
test=pd.read_csv(r'D:\Data\IJCAI\test.csv',na_values=-1)
combine=[train,test]
nantrain,nantest=missing(combine)
nantrain #df[df['user_gender_id'].isnull()]

In [ ]:
nantest

### 分拆item_category_list列,最多三个，第一个相同不用管，先增加一个，然后开始分列

In [ ]:
j=0
for data in combine:
    for ind in data.index:
        if len(data.loc[ind,'item_category_list'].split(';'))==2:
            data.loc[ind,'item_category_list']=data.loc[ind,'item_category_list']+';1000'
        if (ind%10000==0) and (ind!=0):
            print(ind,end=' ')
    #进行分列赋值操作
    data['item_category_list_dup'], data['item_category_list_0'],data['item_category_list_1'] = zip(*data['item_category_list'].map(lambda x: x.split(';')))
train.to_csv(r'D:\Data\IJCAI\train_pre1.csv',index=False)
test.to_csv(r'D:\Data\IJCAI\test_pre1.csv',index=False)

### predict_category_property列里面的属性过多，只考虑类目不考虑属性了，不然行数会增加非常多，而且只使用属性存在的类目！=-1

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\train_pre1.csv',na_values=-1)
test=pd.read_csv(r'D:\Data\IJCAI\test_pre1.csv',na_values=-1)
train.drop(list(train[train['predict_category_property'].isnull()].index),axis=0,inplace=True)
h_=train['predict_category_property'].apply(lambda x: x.split(';'))
h=test['predict_category_property'].apply(lambda x: x.split(';'))
listrain=[]
for i in h_:
    for j in i:
        if j.split(':')[1]!='-1':
            listrain.append(j.split(':')[0])
listest=[]
for i in h:
    for j in i:
        if j.split(':')[1]!='-1':
            listest.append(j.split(':')[0])
combine=[train,test]
######################################################################
for data in combine:
    for i in set(listrain)|set(listest):
        data['predict_category_property_%s'%str(i)]=0
combine=[train,test]
######################################################################
for data in combine:
    for ind in data.index:
        for j in data.loc[ind,'predict_category_property'].split(';'):
            if j.split(':')[1]!='-1':
                data.loc[ind,'predict_category_property_%s'%str(j.split(':')[0])]=1
        if (ind%10000==0) and (ind!=0):
            print(ind,end=' ')
print('Done!')
train.to_csv(r'D:\Data\IJCAI\train_pret.csv',index=False)
test.to_csv(r'D:\Data\IJCAI\test_pret.csv',index=False)

# 对缺失值进行填充

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\train_pret.csv',na_values=-1)
test=pd.read_csv(r'D:\Data\IJCAI\test_pret.csv',na_values=-1)
# corr=train.corr()
corr=pd.read_csv(r'D:\Data\IJCAI\corr.csv',index_col=0)

In [ ]:
print(list(corr[abs(corr['is_trade'])>0.001]['is_trade'].sort_values()[0:27].index))

In [ ]:
combine=[train,test]
nantrain,nantest=missing(combine)
nantrain #df[df['user_gender_id'].isnull()]

In [ ]:
nantest

In [ ]:
corr[corr['shop_review_positive_rate']>0.1]['shop_review_positive_rate'].sort_values(ascending=False).head(6)#shop_score_description
for ind in train[train['shop_review_positive_rate'].isnull()].index:
    item_price_level=train.loc[ind,'item_price_level']
    train.loc[ind,'shop_review_positive_rate']=train[train['item_price_level']==item_price_level]['shop_review_positive_rate'].median()

In [ ]:
corr[corr['shop_score_delivery']>0.1]['shop_score_delivery'].sort_values(ascending=False).head(6)
#['shop_score_delivery','shop_score_description','shop_score_service']这四列是同时缺失的需要一起填充
#train[train['user_age_level'].isnull()][['is_trade','user_age_level','user_occupation_id','user_star_level']]
h=['shop_score_delivery','shop_score_description','shop_score_service']
combine=[train,test]
for data in combine:
    for hj in h:
        for ind in data[data[hj].isnull()].index:
            shop_review_positive_rate=data.loc[ind,'shop_review_positive_rate']
            item_price_level=data.loc[ind,'item_price_level']
            temp=data[(data['shop_review_positive_rate']==shop_review_positive_rate)&(data['item_price_level']==item_price_level)]
            if temp['shop_score_service'].isnull().any():
                data.loc[ind,'shop_score_service']=data[data['item_price_level']==item_price_level]['shop_score_service'].median()
            else:
                data.loc[ind,'shop_score_service']=train.median()
            if temp['shop_score_delivery'].isnull().any():
                data.loc[ind,'shop_score_delivery']=data[data['item_price_level']==item_price_level]['shop_score_delivery'].median()
            else:
                data.loc[ind,'shop_score_delivery']=round(train['shop_score_delivery'].median(),6)
            if temp['shop_score_description'].isnull().any():
                data.loc[ind,'shop_score_description']=data[data['item_price_level']==item_price_level]['shop_score_description'].median()
            else:
                data.loc[ind,'shop_score_description']=train['shop_score_description'].median()

In [ ]:
corr[corr['user_age_level']>0.1]['user_age_level'].sort_values(ascending=False).head(6)
#['is_trade','user_age_level','user_occupation_id','user_star_level']这三列是同时缺失的需要一起填充
combine=[train,test]
h=['user_age_level','user_occupation_id','user_star_level']
for data in combine:
    for hj in h:
        for ind in data[data[hj].isnull()].index:
            shop_score_description=data.loc[ind,'shop_score_description']
            item_price_level=data.loc[ind,'item_price_level']
            temp=data[(data['shop_score_description']==shop_score_description)&(data['item_price_level']==item_price_level)]
            if temp['user_age_level'].isnull().any():
                data.loc[ind,'user_age_level']=data[data['item_price_level']==item_price_level]['user_age_level'].median()
            else:
                data.loc[ind,'user_age_level']=temp['user_age_level'].median()
            if temp['user_occupation_id'].isnull().any():
                if (data[data['shop_score_description']==shop_score_description]['user_occupation_id'].isnull().any())\
                and (data[data['item_price_level']==item_price_level]['user_occupation_id'].isnull().any()):
                    data.loc[ind,'user_occupation_id']=data.loc[ind+6,'user_occupation_id']
                elif (data[data['shop_score_description']==shop_score_description]['user_occupation_id'].isnull().any())\
                and (not data[data['item_price_level']==item_price_level]['user_occupation_id'].isnull().any()):
                    data.loc[ind,'user_occupation_id']=data[data['item_price_level']==item_price_level]['user_occupation_id'].value_counts().index[0]
                else:
                    data.loc[ind,'user_occupation_id']=data[data['shop_score_description']==shop_score_description]['user_occupation_id'].value_counts().index[0]
            else:
                data.loc[ind,'user_occupation_id']=temp['user_occupation_id'].value_counts().index[0]
            if temp['user_star_level'].isnull().any():
                data.loc[ind,'user_star_level']=data[data['shop_score_description']==shop_score_description]['user_star_level'].median()
            else:
                data.loc[ind,'user_star_level']=temp['user_star_level'].median()
    for ind in data[data['user_star_level'].isnull()].index:
        data.loc[ind,'user_star_level']=data.loc[ind-6,'user_star_level']
    for ind in data[data['user_age_level'].isnull()].index:
        data.loc[ind,'user_age_level']=data.loc[ind-6,'user_age_level']

In [ ]:
corr[corr['item_brand_id']>0.06]['item_brand_id'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    for ind in data[data['item_brand_id'].isnull()].index:
        shop_id=data.loc[ind,'shop_id']
        item_category_list_0=data.loc[ind,'item_category_list_0']
        temp=data[(data['shop_id']==shop_id)&(data['item_category_list_0']==item_category_list_0)]
        if len(temp['item_brand_id'].isnull())==len(temp):
            data.loc[ind,'item_brand_id']=data.loc[ind-1,'item_brand_id']
        else:
            data.loc[ind,'item_brand_id']=temp['item_brand_id'].value_counts().index[0]

In [ ]:
corr[corr['item_city_id']>0.006]['item_city_id'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    for ind in data[data['item_city_id'].isnull()].index:
        item_price_level=data.loc[ind,'item_price_level']
        predict_category_property_5755694407684602296=data.loc[ind,'predict_category_property_5755694407684602296']
        temp=data[(data['item_price_level']==item_price_level)&(data['predict_category_property_5755694407684602296']==predict_category_property_5755694407684602296)]
        data.loc[ind,'item_city_id']=temp['item_city_id'].value_counts().index[0]

In [ ]:
# corr[corr['item_sales_level']>0.1]['item_sales_level'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    for ind in data[data['item_sales_level'].isnull()].index:
        item_collected_level=data.loc[ind,'item_collected_level']
        item_pv_level=data.loc[ind,'item_pv_level']
        temp=data[(data['item_collected_level']==item_collected_level)&(data['item_pv_level']==item_pv_level)]
        if temp['item_sales_level'].isnull().any():
            data.loc[ind,'item_sales_level']=data[data['item_collected_level']==item_collected_level]['item_sales_level'].median()
        else:
            data.loc[ind,'item_sales_level']=temp['item_sales_level'].median()

In [ ]:
corr[corr['user_gender_id']>0.01]['user_gender_id'].sort_values(ascending=False).head(6)
combine=[train,test]
for data in combine:
    count=0
    for ind in data[data['user_gender_id'].isnull()].index:
        count+=1
        user_age_level=data.loc[ind,'user_age_level']
        predict_category_property_6007511169620956394=data.loc[ind,'predict_category_property_6007511169620956394']
        temp=data[(data['user_age_level']==user_age_level)&(data['predict_category_property_6007511169620956394']==predict_category_property_6007511169620956394)]
        if (len(temp)==0) and (len(data[data['user_age_level']==user_age_level]['user_gender_id'])!=0):
            data.loc[ind,'user_gender_id']=data[data['user_age_level']==user_age_level]['user_gender_id'].value_counts().index[0]
        elif (len(temp)==0) and (len(data[data['user_age_level']==user_age_level]['user_gender_id'])==0):
            data.loc[ind,'user_gender_id']=data[data['predict_category_property_6007511169620956394']==predict_category_property_6007511169620956394]['user_gender_id'].value_counts().index[0]
        elif temp['user_gender_id'].isnull().any():
            data.loc[ind,'user_gender_id']=data['user_gender_id'].value_counts().index[0]
        else:
            data.loc[ind,'user_gender_id']=temp['user_gender_id'].value_counts().index[0]
        if count%1000==0:
            print(count,end=' ')

### 提取前一天、前两天的转换率

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\train_mispret.csv')#,na_values=-1 [160955, 205287, 214183, 314706]
test=pd.read_csv(r'D:\Data\IJCAI\test_mispret.csv')
combine=[test]
train['timestamp']=pd.to_datetime(train['context_timestamp'].copy(),unit='s')
train['day']=train['timestamp'].dt.day
train['num_conversion']=0
train['num_conversiontd']=0
for day in set(train['day'].values):
    if day<19:
        continue
    else:
        for u in set(train[train['day']==day]['item_id'].values):
            if u in train[train['day']==(day-1)]['item_id'].values:
                train.loc[train[(train['item_id']==u)&(train['day']==day)].index,'num_conversion']=\
                len(train[(train['item_id']==u)&(train['day']==(day-1))&(train['is_trade']==1)])/len(train[(train['item_id']==u)&(train['day']==(day-1))])
            if u in train[(train['day']==(day-1))|(train['day']==(day-2))]['item_id'].values:
                train.loc[train[(train['item_id']==u)&(train['day']==day)].index,'num_conversiontd']=\
                (len(train[(train['item_id']==u)&(train['day']==(day-1))&(train['is_trade']==1)])+\
                 len(train[(train['item_id']==u)&(train['day']==(day-2))&(train['is_trade']==1)]))/(len(train[(train['item_id']==u)&(train['day']==(day-1))])+\
                                                                                                    len(train[(train['item_id']==u)&(train['day']==(day-2))]))
combine=[test]
for data in combine:
    data['num_conversion']=0
    for u in set(data['item_id'].values):
        if u in train[train['day']==24]['item_id'].values:
            data.loc[data[data['item_id']==u].index,'num_conversion']=\
            len(train[(train['item_id']==u)&(train['day']==24)&(train['is_trade']==1)])/len(train[(train['item_id']==u)&(train['day']==24)])
        if u in train[(train['day']==24)|(train['day']==23)]['item_id'].values:
            data.loc[data[data['item_id']==u].index,'num_conversiontd']=\
            (len(train[(train['item_id']==u)&(train['day']==24)&(train['is_trade']==1)])+\
             len(train[(train['item_id']==u)&(train['day']==23)&(train['is_trade']==1)]))/(len(train[(train['item_id']==u)&(train['day']==24)])+\
                                                                                                len(train[(train['item_id']==u)&(train['day']==23)]))

train.drop(['timestamp','day'],axis=1,inplace=True)
train.to_csv(r'D:\Data\IJCAI\train_mispret.csv',index=False)
test.to_csv(r'D:\Data\IJCAI\test_mispret.csv',index=False)

## 提取每天用户点击数，商品被点击数，各个时间间隔,并将train和test重新分开，test按照最开始的顺序排序

In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv(r'D:\Data\IJCAI\train_mispret.csv')#,na_values=-1 [160955, 205287, 214183, 314706]
test=pd.read_csv(r'D:\Data\IJCAI\test_mispret.csv')
train['timestamp']=pd.to_datetime(train['context_timestamp'].copy(),unit='s')
test['timestamp']=pd.to_datetime(test['context_timestamp'].copy(),unit='s')
train['day']=train['timestamp'].dt.day
test['day']=test['timestamp'].dt.day
train['num_click']=0
test['num_click']=0
train['num_show']=0
test['num_show']=0

train=train.sort_values(by=['day','user_id'])                               #同一天同一个用户的点击广告的次数——--用户的搜索次数和点击次数
h=train.groupby(by=['day','user_id']).count()
lis=[]
for i,j in h.index:
    lis.extend([h.loc[i].loc[j]['user_gender_id']]*h.loc[i].loc[j]['user_gender_id'])
train['num_click']=lis

train=train.sort_values(by=['day','item_id'])                              #同一天同一件商品被搜索点击次数-----商品的被搜索次数和受欢迎程度
h=train.groupby(by=['day','item_id']).count()
lis=[]
for i,j in h.index:
    lis.extend([h.loc[i].loc[j]['user_gender_id']]*h.loc[i].loc[j]['user_gender_id'])
train['num_show']=lis
    
train=train.sort_values(by=['item_id','context_timestamp'])                 #同一件商品被搜索点击的时间间隔
train.index=np.arange(len(train))
lis=[]
for val in set(train['item_id'].values):
    lis.append(train[train['item_id']==val].index[-1])
temp=train['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
train['item_time_diff']=temp

train=train.sort_values(by=['day','item_id','context_timestamp'])          #同一天同一件商品的被搜索点击的时间间隔
train.index=np.arange(len(train))
lis=[]
for d in [17,18,19,20,21,22,23,24]:
    for val in set(train[train['day']==d]['item_id'].values):
        lis.append(train[(train['item_id']==val)&(train['day']==d)].index[-1])
temp=train['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
train['day_item_diff']=temp
    
train=train.sort_values(by=['day','user_id','context_timestamp'])          #同一天相同用户的不同商品点击时间间隔
train.index=np.arange(len(train))
lis=[]
for d in [17,18,19,20,21,22,23,24]:
    for val in set(train[train['day']==d]['user_id'].values):
        lis.append(train[(train['user_id']==val)&(train['day']==d)].index[-1])
temp=train['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
train['day_user_diff']=temp
train.drop(['timestamp','day'],axis=1,inplace=True)
train.to_csv(r'D:\Data\IJCAI\train_mispret.csv',index=False)
print('Train has being Done!')
##########################################################################################################################################
test=test.sort_values(by=['day','user_id'])                               #同一天同一个用户的点击广告的次数——--用户的搜索次数和点击次数
h=test.groupby(by=['day','user_id']).count()
lis=[]
for i,j in h.index:
    lis.extend([h.loc[i].loc[j]['user_gender_id']]*h.loc[i].loc[j]['user_gender_id'])
test['num_click']=lis

test=test.sort_values(by=['day','item_id'])                              #同一天同一件商品被搜索点击次数-----商品的被搜索次数和受欢迎程度
h=test.groupby(by=['day','item_id']).count()
lis=[]
for i,j in h.index:
    lis.extend([h.loc[i].loc[j]['user_gender_id']]*h.loc[i].loc[j]['user_gender_id'])
test['num_show']=lis
    
test=test.sort_values(by=['item_id','context_timestamp'])                 #同一件商品被搜索点击的时间间隔
test.index=np.arange(len(test))
lis=[]
for val in set(test['item_id'].values):
    lis.append(test[test['item_id']==val].index[-1])
temp=test['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
test['item_time_diff']=temp

test=test.sort_values(by=['day','item_id','context_timestamp'])          #同一天同一件商品的被搜索点击的时间间隔
test.index=np.arange(len(test))
lis=[]
for d in [24,25]:
    for val in set(test[test['day']==d]['item_id'].values):
        lis.append(test[(test['item_id']==val)&(test['day']==d)].index[-1])
temp=test['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
test['day_item_diff']=temp
    
test=test.sort_values(by=['day','user_id','context_timestamp'])          #同一天相同用户的不同商品点击时间间隔
test.index=np.arange(len(test))
lis=[]
for d in [24,25]:
    for val in set(test[test['day']==d]['user_id'].values):
        lis.append(test[(test['user_id']==val)&(test['day']==d)].index[-1])
temp=test['context_timestamp'].diff()
for i in lis:
    temp.loc[i+1]=0
temp.loc[0]=0
test['day_user_diff']=temp

test.drop(['timestamp','day'],axis=1,inplace=True)
test.to_csv(r'D:\Data\IJCAI\test_mispret.csv',index=False)

import pandas as pd
import numpy as np
test=pd.read_csv(r'D:\Data\IJCAI\test_mispret.csv')
new=pd.read_csv(r'D:\Data\IJCAI\test.csv')
lis=[]
for i in new.index:
    lis.extend(list(test[test['instance_id']==new.loc[i,'instance_id']].index))
test=test.loc[lis]
test.index=np.arange(len(test))
test.to_csv(r'D:\Data\IJCAI\test_mispret.csv',index=False)

In [1]:
import pandas as pd
import numpy as np
import math
tr=pd.read_csv(r'D:\Data\IJCAI\train_pret.csv',usecols=['instance_id','item_property_list','item_category_list','predict_category_property'])
tr.drop_duplicates(inplace=True)
train=pd.read_csv(r'D:\Data\IJCAI\train_mispret.csv')#,na_values=-1 [160955, 205287, 214183, 314706]
test=pd.read_csv(r'D:\Data\IJCAI\test_mispret.csv')
te=pd.read_csv(r'D:\Data\IJCAI\test_pret.csv')

In [2]:
test=pd.concat([test,te[['item_property_list','item_category_list','predict_category_property']]],axis=1)
train=train.sort_values(by=['instance_id'])
tr=tr.sort_values(by=['instance_id'])
tr.index=np.arange(len(tr))
train.index=np.arange(len(train))
train=pd.concat([train,tr[['item_property_list','item_category_list','predict_category_property']]],axis=1)

## 广告商品和预测属性和类目数量和比率

In [3]:
import time 
s=time.time()
combine=[train,test]
for data in combine:
    data['com_pre_cate']=0
    data['com_pre_pro']=0
    data['com_pre_cateprob']=0
    data['com_pre_proprob']=0
    data['num_true_category']=0
    data['num_true_property']=0
    data['num_pred_category']=0
    data['num_pred_property']=0
    for ind in data.index:
        Category=data.loc[ind,'item_category_list'].split(';')
        Property=data.loc[ind,'item_property_list'].split(';')
        liscate=[]
        lispro=[]
        for i in data.loc[ind,'predict_category_property'].split(';'):
            liscate.append(i.split(':')[0])
            lispro.extend(i.split(':')[1].split(','))
        while '1000' in Category:
            Category.remove('1000')
        while '-1' in lispro:
            lispro.remove('-1')
        comcate=len(set(Category)&set(liscate))
        compro=len(set(Property)&set(lispro))
        data.loc[ind,['com_pre_cate','com_pre_pro','com_pre_cateprob','com_pre_proprob','num_true_category','num_true_property',\
                      'num_pred_category','num_pred_property']]=[comcate,compro,comcate/math.sqrt(len(Category)*len(liscate)+1),\
                                                                 compro/math.sqrt(len(Property)*len(lispro)+1),len(Category),len(Property),\
                                                                 len(liscate),len(lispro)]
        if (ind %10000==0) and (ind!=0):
            print(ind,end=' ')
e=time.time()
print(e-s)
train.to_csv(r'D:\Data\IJCAI\train_mispret.csv',index=False)
test.to_csv(r'D:\Data\IJCAI\test_mispret.csv',index=False)

10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 10000 47824.31797790527
